# Walmart Kaggle Project

### Authors:  Shijie Wang and Matthew Odom

In [2]:
%ls

Walmart_Script.ipynb   test.csv               train.csv
sample_submission.csv  test.csv.zip


In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import random

# load the datasets
train = pd.read_csv("train.csv")
train = pd.get_dummies(train)
train = train.dropna()

test = pd.read_csv("test.csv")
test = pd.get_dummies(test)
test = test.dropna()

# subset for algorithm build
data = train[train["TripType"] <= 10]
data = data.sample(50000)

# clean dataset for modeling data.ix[random.sample(data.index, 50000)]
# create a loop for each TripType
# stack the output of the test

TripTypes = data.TripType.unique()
submission = pd.DataFrame()

for i in TripTypes:
    print "Working on TripType " + str(i)
    
    # reset dataset for next iteration
    sample = data
    sample = sample.dropna()
    
    # convert independent variable into binary
    sample.TripType = np.where(sample.TripType != i, 0, 1) 
    
    # set up model
    X = sample.ix[:,sample.columns != 'TripType']
    y = sample.ix[:,sample.columns == 'TripType']
    model = sm.OLS(y,X)
    results = model.fit()
    
    # predict test dataset values
    yhat = pd.DataFrame(results.predict(X)) # this should be results.predict(test)
    yhat.columns = ["TripType_%s" % i]
    
    # append columns to submission
    submission = submission.join(yhat, how = "outer")
    
submission

Working on TripType 7
Working on TripType 8
Working on TripType 6
Working on TripType 5
Working on TripType 9
Working on TripType 3
Working on TripType 4


,TripType_7,TripType_8,TripType_6,TripType_5,TripType_9,TripType_3,TripType_4
0,0.585010,0.306958,0.014576,0.071882,0.022890,0.000881,-0.002197
1,0.021126,0.655246,0.001518,0.218511,0.082765,0.014638,0.006193
2,0.121574,0.258398,0.435515,0.078269,0.080932,0.019529,0.005785
3,0.655926,0.257108,0.010630,0.049290,0.020177,0.001343,0.005527
4,0.523815,0.351788,0.020564,0.075289,0.020685,0.000097,0.007762
5,0.028308,0.126420,0.034936,0.084001,0.706725,0.011220,0.008410
6,0.906272,0.046849,0.013881,0.024585,0.008887,-0.000067,-0.000404
7,0.017971,0.666831,0.011770,0.212663,0.081722,0.003064,0.005978
8,0.384572,0.437799,0.070078,0.062814,0.044969,0.004232,-0.004460
9,0.255643,0.526828,0.060252,0.075476,0.062053,0.012634,0.007115


In [57]:
submission = pd.DataFrame()
i=5
sample = data
sample = sample.dropna()

# convert independent variable into binary
sample.TripType = np.where(sample.TripType != i, 0, 1) 

# set up model
X = sample.ix[:,sample.columns != 'TripType']
y = sample.ix[:,sample.columns == 'TripType']
model = sm.OLS(y,X)
results = model.fit()

# predict test dataset values
yhat = results.predict(X)

# append columns to submission
submission = submission.join(pd.DataFrame(yhat), how = "outer")

#pd.concat([submission, pd.DataFrame(yhat)], axis=1)
submission

,0
0,0.762925
1,0.134669
2,-0.031365
3,0.071226
4,0.025883
5,0.063250
6,0.057436
7,0.058333
8,0.059009
9,0.067904


In [151]:
print results.summary()

                            OLS Regression Results                            
Dep. Variable:               TripType   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     378.1
Date:                Sat, 21 Nov 2015   Prob (F-statistic):               0.00
Time:                        22:47:28   Log-Likelihood:                -5161.5
No. Observations:               48456   AIC:                         1.048e+04
Df Residuals:                   48379   BIC:                         1.115e+04
Df Model:                          76                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------